In [1]:
# https://cran.r-project.org/web/packages/data.table/data.table.pdf
# 載入套件
library("data.table")

Warning message:
"package 'data.table' was built under R version 3.5.3"

In [2]:
# 建立一data.frame資料
name <- c("Joe", "Bob", "Vicky")
age <- c("28", "26", "34")
gender <- c("Male","Male","Female")
(data <- data.frame(name, age, gender, row.names=c("a","b","c")))

,name,age,gender
,<fct>,<fct>,<fct>
a,Joe,28,Male
b,Bob,26,Male
c,Vicky,34,Female


In [3]:
# data.table不包含rownames
(x1 <- data.table(data))

cat("\n")

#可設置參數keep.rownames = TRUE, 新增一行rn來表示rownames
(x1 <- data.table(data, keep.rownames = TRUE))

name,age,gender
<fct>,<fct>,<fct>
Joe,28,Male
Bob,26,Male
Vicky,34,Female


rn,name,age,gender
<chr>,<fct>,<fct>,<fct>
a,Joe,28,Male
b,Bob,26,Male
c,Vicky,34,Female


In [4]:
# data.table預設stringsAsFactors為FALSE, 但從data.frame轉換來的依然有Factor
x2 <- data.table(name, age, gender)
class(data$name)
class(x1$name)
class(x2$name)

[1] "factor"

[1] "factor"

[1] "character"

In [5]:
# := 不產生副本的情況下, 增減column
x2$age <- as.numeric(x2$age)
x2[,stupid := 87]
x2

cat("\n")

# x2[,c("age.dev", "age.sq") := list(age-mean(age),(age.dev)^2)] 
# 會出現error,只能使用已賦值的變數

x2[,c("age.dev", "age.sq") := list(age-mean(age),(age-mean(age))^2)]
x2

cat("\n")

# 復原
x2[,c("stupid", "age.dev",  "age.sq") := NULL]
x2

name,age,gender,stupid
<chr>,<dbl>,<chr>,<dbl>
Joe,28,Male,87
Bob,26,Male,87
Vicky,34,Female,87


name,age,gender,stupid,age.dev,age.sq
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
Joe,28,Male,87,-1.333333,1.777778
Bob,26,Male,87,-3.333333,11.111111
Vicky,34,Female,87,4.666667,21.777778


name,age,gender
<chr>,<dbl>,<chr>
Joe,28,Male
Bob,26,Male
Vicky,34,Female


In [6]:
# setDT, setDF 不產生副本的情況下,相互轉換data.table跟data.frame
print("data.frame轉成data.table")
setDT(data, keep.rownames = TRUE)
class(data)
show(data)

cat("\n")

print("復原後")
setDF(data, rownames = data$rn)
data <- data[,-1]
class(data)
data

[1] "data.frame轉成data.table"


[1] "data.table" "data.frame"

   rn  name age gender
1:  a   Joe  28   Male
2:  b   Bob  26   Male
3:  c Vicky  34 Female

[1] "復原後"


[1] "data.frame"

,name,age,gender
,<fct>,<fct>,<fct>
a,Joe,28,Male
b,Bob,26,Male
c,Vicky,34,Female


In [7]:
(DT = data.table(x = rep(c("b","a","c"),each=3), v = c(1,1,1,2,2,1,1,2,2), y = c(1,3,6), a = 1:9, b = 9:1))
(X = data.table(x = c("c","b","b"), v = 8:6, foo = c(4,2,7)))

x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
b,1,6,3,7
a,2,1,4,6
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


x,v,foo
<chr>,<int>,<dbl>
c,8,4
b,7,2
b,6,7


In [8]:
# 取列子集

DT[3:4]
DT[-(3:4)]            # 同 DT[!3:4]            
DT[order(y)]
DT[a > 3 & y == 6]

x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,6,3,7
a,2,1,4,6


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
a,2,1,4,6
c,1,1,7,3
b,1,3,2,8
a,2,3,5,5
c,2,3,8,2
b,1,6,3,7
a,1,6,6,4
c,2,6,9,1


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
a,1,6,6,4
c,2,6,9,1


In [9]:
DT[, x]         # 回傳為vector
DT[, .(x)]      # 回傳為data.table

[1] "b" "b" "b" "a" "a" "a" "c" "c" "c"

x
<chr>
b
b
b
a
a
a
c
c
c


In [10]:
# by, keyby  分組運算
DT[, sum(v), by = x]
DT[, sum(v), keyby = x]   # 會將組別排序, 等同於DT[, sum(v), by = x][order(x)]

x,V1
<chr>,<dbl>
b,3
a,5
c,5


x,V1
<chr>,<dbl>
a,5
b,3
c,5


In [11]:
# keyby 多變數的情況
DT[, sum(a), keyby = "x,v"]

x,v,V1
<chr>,<dbl>,<int>
a,1,6
a,2,9
b,1,6
c,1,7
c,2,17


In [12]:
# 先篩選列, 再根據組別做運算
DT[a < 6, sum(y), by = x]   

x,V1
<chr>,<dbl>
b,10
a,4


In [13]:
# match data
# 若first arguement為另一個data, 則可用on作為match資料的依據

DT[X, .(a, b), on = "x"]  # 取得DT$x == X$x的列, 再取其a,b的值
DT[!X, .(a, b), on = "x"]  # 取得DT$x != X$x的列, 再取其a,b的值

DT[X, .(a, i.v, x), on = "x"]    # 取得DT$x == X$x的列, 再取 DT$a, X$v, x 的值
cat("\n\n\n\n")

DT[X, .(v, i.v, x), on = "x"]

cat("計算總和為:")
DT[X, sum(v, i.v), on = "x"]
cat("\n\n")

cat("根據X$x個別計算, 總合為:
")
DT[X, sum(v, i.v), on = "x", by = .EACHI]    # 根據X$x個別做運算, 每個i.v只計算一次

a,b
<int>,<int>
7,3
8,2
9,1
1,9
2,8
3,7
1,9
2,8
3,7


a,b
<int>,<int>
4,6
5,5
6,4


a,i.v,x
<int>,<int>,<chr>
7,8,c
8,8,c
9,8,c
1,7,b
2,7,b
3,7,b
1,6,b
2,6,b
3,6,b


v,i.v,x
<dbl>,<int>,<chr>
1,8,c
2,8,c
2,8,c
1,7,b
1,7,b
1,7,b
1,6,b
1,6,b
1,6,b


計算總和為:

[1] 74



根據X$x個別計算, 總合為:


x,V1
<chr>,<dbl>
c,13
b,10
b,9


In [14]:
# 若first arguement為元素值, 依on所指示之變數回傳相符的資料點
DT
DT["b", on = "x"]      # 為二進位搜尋比較一般快
DT[.(2), on = .(v)]    # .()為list()的縮寫, 較方便, 不須加引號

# 尋找x = "b", 且y=3或y=4或y=5或y=6
DT[.("b", 3:6), on = .(x, y)]                     # no match, 則回傳NA

# 尋找x!="b", 或 y!=3且y!=4且y!=5且y!=6
DT[!.("b", 3:6), on = .(x, y)]                    # 與 DT[x != "b" | !y%in%c(3:6)] 相等

DT[.("b", 3:6), on = .(x, y), nomatch = FALSE]    # no match, 則不回傳
DT[.("b", 3:6), on = .(x, y), roll = Inf]         # no match, 則代換為前一筆資料的值
DT[.("b", 3:6), on = .(x, y), roll = -Inf]        # no match, 則代換為下一筆資料的值
DT[.("b", 3:6), on = .(x, y), roll = "nearest"]   # no match, 則代換為最鄰近資料的值

x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
b,1,6,3,7
a,2,1,4,6
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
b,1,6,3,7


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
a,2,1,4,6
a,2,3,5,5
c,2,3,8,2
c,2,6,9,1


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,3,2,8
b,NA,4,NA,NA
b,NA,5,NA,NA
b,1,6,3,7


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
a,2,1,4,6
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,3,2,8
b,1,6,3,7


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,3,2,8
b,1,4,2,8
b,1,5,2,8
b,1,6,3,7


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,3,2,8
b,1,4,3,7
b,1,5,3,7
b,1,6,3,7


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,3,2,8
b,1,4,2,8
b,1,5,3,7
b,1,6,3,7


In [15]:
# .N 關於列數
DT
DT[.N]              # 最後一列, 是唯一可以在"列"使用的特殊符號
DT[, .N]            # 列的總數
DT[, .N, by=x]      # x中各群的列數

x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
b,1,6,3,7
a,2,1,4,6
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
c,2,6,9,1


[1] 9

x,N
<chr>,<int>
b,3
a,3
c,3


In [16]:
# .SD 關於行列選取
DT
DT[, .SD, .SDcols = x:y]                 # 選擇x跟y及它們之間所有的"行"
DT[, .SD, .SDcols = !x:y]                # 排除x跟y及它們之間所有的"行"
DT[, .SD, .SDcols = patterns('^[xv]')]   # 選擇行名含有'x'或'v'的行
DT[, .SD[1]]                             # 所有行的第一列
DT[, .SD[1, .(y, v)], by = x]            # 以x分組, 找個組y,v的第一列
DT[, c(.N, lapply(.SD, sum)), by=x]      # 以X分群, 計算列數並計算各行總和

x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
b,1,6,3,7
a,2,1,4,6
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


x,v,y
<chr>,<dbl>,<dbl>
b,1,1
b,1,3
b,1,6
a,2,1
a,2,3
a,1,6
c,1,1
c,2,3
c,2,6


a,b
<int>,<int>
1,9
2,8
3,7
4,6
5,5
6,4
7,3
8,2
9,1


x,v
<chr>,<dbl>
b,1
b,1
b,1
a,2
a,2
a,1
c,1
c,2
c,2


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9


x,y,v
<chr>,<dbl>,<dbl>
b,1,1
a,1,2
c,1,1


x,N,v,y,a,b
<chr>,<int>,<dbl>,<dbl>,<int>,<int>
b,3,3,10,6,24
a,3,5,10,15,15
c,3,5,10,24,6


In [17]:
# .I 關於row number
DT[, .I]              # 各資料點的row number
DT[, .I[1], by = x]   # 以x分組, 找各組第一個資料點的row number

[1] 1 2 3 4 5 6 7 8 9

x,V1
<chr>,<int>
b,1
a,4
c,7


In [18]:
# .GRP
DT[, g := .GRP, by = y]    # 以y分組, 變數g標註該資料來自y的第幾群
DT

# 復原
DT[, g := NULL]
DT

x,v,y,a,b,g
<chr>,<dbl>,<dbl>,<int>,<int>,<int>
b,1,1,1,9,1
b,1,3,2,8,2
b,1,6,3,7,3
a,2,1,4,6,1
a,2,3,5,5,2
a,1,6,6,4,3
c,1,1,7,3,1
c,2,3,8,2,2
c,2,6,9,1,3


x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
b,1,6,3,7
a,2,1,4,6
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


In [19]:
#.BY 為已分群的值組成的list
DT
X
X[, DT[.BY, .(v,y), on = "x"], by = x]  # 因by = x, 所以此處.BY等於.(c,b)

x,v,y,a,b
<chr>,<dbl>,<dbl>,<int>,<int>
b,1,1,1,9
b,1,3,2,8
b,1,6,3,7
a,2,1,4,6
a,2,3,5,5
a,1,6,6,4
c,1,1,7,3
c,2,3,8,2
c,2,6,9,1


x,v,foo
<chr>,<int>,<dbl>
c,8,4
b,7,2
b,6,7


x,v,y
<chr>,<dbl>,<dbl>
c,1,1
c,2,3
c,2,6
b,1,1
b,1,3
b,1,6


In [20]:
data(Cars93, package="MASS")
x3 <- data.table(Cars93[,c("Manufacturer","Type","Price","Length","Weight","Origin")])
x3

Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
Acura,Small,15.9,177,2705,non-USA
Acura,Midsize,33.9,195,3560,non-USA
Audi,Compact,29.1,180,3375,non-USA
Audi,Midsize,37.7,193,3405,non-USA
BMW,Midsize,30.0,186,3640,non-USA
Buick,Midsize,15.7,189,2880,USA
Buick,Large,20.8,200,3470,USA
Buick,Large,23.7,216,4105,USA
Buick,Midsize,26.3,198,3495,USA


In [25]:
# key 快速搜尋及篩選
setkey(x3, Origin)                    
cat("x3的key為:", key(x3), "\n\n")     
tables()                              # 概述環境下的data.table
x3["USA"]


x3的key為: Origin 

   NAME NROW NCOL MB                                         COLS    KEY
1:   DT    9    5  0                                    x,v,y,a,b       
2:    X    3    3  0                                      x,v,foo       
3:   x1    3    4  0                           rn,name,age,gender       
4:   x2    3    3  0                              name,age,gender       
5:   x3   93    6  0 Manufacturer,Type,Price,Length,Weight,Origin Origin
Total: 0MB


Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
Buick,Midsize,15.7,189,2880,USA
Buick,Large,20.8,200,3470,USA
Buick,Large,23.7,216,4105,USA
Buick,Midsize,26.3,198,3495,USA
Cadillac,Large,34.7,206,3620,USA
Cadillac,Midsize,40.1,204,3935,USA
Chevrolet,Compact,13.4,182,2490,USA
Chevrolet,Compact,11.4,184,2785,USA
Chevrolet,Sporty,15.1,193,3240,USA


In [49]:
setkey(x3, Origin, Type)
key(x3)
x3[J("USA", c("Large", "Midsize"))]   # 用J函數呼叫多個key
x3[J("Large", "USA")]                 # 順序有要求
x3["USA"]                             # 如果不呼叫全部的key, 則會依序處理, 此處適用的key是Origin
x3["Large"]                           # 變數Origin並沒有Large這個元素
x3[J(c("non-USA", "USA"), c("Midsize", "Large"))]    # 2種組合
x3[CJ(c("USA", "non-USA"), c("Large", "Midsize"))]   # cross join: 4種組合

[1] "Origin" "Type"

Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
Buick,Large,20.8,200,3470,USA
Buick,Large,23.7,216,4105,USA
Cadillac,Large,34.7,206,3620,USA
Chevrolet,Large,18.8,214,3910,USA
Chrylser,Large,18.4,203,3515,USA
Chrysler,Large,29.5,203,3570,USA
Eagle,Large,19.3,202,3490,USA
Ford,Large,20.9,212,3950,USA
Lincoln,Large,36.1,219,4055,USA


Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
NA,USA,NA,NA,NA,Large


Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
Chevrolet,Compact,13.4,182,2490,USA
Chevrolet,Compact,11.4,184,2785,USA
Chrysler,Compact,15.8,183,3085,USA
Dodge,Compact,13.3,181,2970,USA
Ford,Compact,11.3,177,2690,USA
Oldsmobile,Compact,13.5,188,2910,USA
Pontiac,Compact,11.1,181,2575,USA
Buick,Large,20.8,200,3470,USA
Buick,Large,23.7,216,4105,USA


Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
NA,NA,NA,NA,NA,Large


Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
Buick,Large,20.8,200,3470,USA
Buick,Large,23.7,216,4105,USA
Cadillac,Large,34.7,206,3620,USA
Chevrolet,Large,18.8,214,3910,USA
Chrylser,Large,18.4,203,3515,USA
Chrysler,Large,29.5,203,3570,USA
Eagle,Large,19.3,202,3490,USA
Ford,Large,20.9,212,3950,USA
Lincoln,Large,36.1,219,4055,USA


Manufacturer,Type,Price,Length,Weight,Origin
<fct>,<fct>,<dbl>,<int>,<int>,<fct>
Buick,Large,20.8,200,3470,USA
Buick,Large,23.7,216,4105,USA
Cadillac,Large,34.7,206,3620,USA
Chevrolet,Large,18.8,214,3910,USA
Chrylser,Large,18.4,203,3515,USA
Chrysler,Large,29.5,203,3570,USA
Eagle,Large,19.3,202,3490,USA
Ford,Large,20.9,212,3950,USA
Lincoln,Large,36.1,219,4055,USA
